# Basig usage of qmrExchange

### Import required libraries

In [1]:
import sys
sys.path.append('..')
from main import *   

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
time_interval = 'minute'
tickers = ['XYZ']
episodes = 1000

### Instantiate a Simulator

In [3]:
sim = Simulator(time_unit=time_interval, episodes=episodes)
sim.exchange.create_asset(tickers[0])


### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = NaiveMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = RandomMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

print(sim.agents)

[<Agent: market_maker>, <Agent: market_taker>]


### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [6]:
print(sim.exchange.trade_log)
# print(sim.trades.head(20).to_markdown())

[<Trade: XYZ 1@100 2022-01-01 00:00:00>, <Trade: XYZ 1@100.25 2022-01-01 00:02:00>, <Trade: XYZ 1@100.0 2022-01-01 00:05:00>, <Trade: XYZ 0@99.75 2022-01-01 00:08:00>, <Trade: XYZ 1@100.0 2022-01-01 00:12:00>, <Trade: XYZ 1@99.75 2022-01-01 00:15:00>, <Trade: XYZ 0@99.5 2022-01-01 00:17:00>, <Trade: XYZ 0@99.25 2022-01-01 00:18:00>, <Trade: XYZ 0@99.0 2022-01-01 00:34:00>, <Trade: XYZ 1@99.25 2022-01-01 00:35:00>, <Trade: XYZ 1@99.5 2022-01-01 00:36:00>, <Trade: XYZ 2@99.25 2022-01-01 00:37:00>, <Trade: XYZ 1@99.5 2022-01-01 00:42:00>, <Trade: XYZ 1@99.75 2022-01-01 00:43:00>, <Trade: XYZ 1@100.0 2022-01-01 00:45:00>, <Trade: XYZ 1@100.25 2022-01-01 00:46:00>, <Trade: XYZ 4@100.0 2022-01-01 00:48:00>, <Trade: XYZ 1@100.25 2022-01-01 00:49:00>, <Trade: XYZ 1@100.0 2022-01-01 00:51:00>, <Trade: XYZ 0@99.75 2022-01-01 00:52:00>, <Trade: XYZ 0@99.5 2022-01-01 00:56:00>, <Trade: XYZ 0@99.25 2022-01-01 00:59:00>, <Trade: XYZ 1@99.5 2022-01-01 01:04:00>, <Trade: XYZ 1@99.25 2022-01-01 01:06:0

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:05:00', '2022-01-01 00:08:00',
               '2022-01-01 00:12:00', '2022-01-01 00:15:00',
               '2022-01-01 00:17:00', '2022-01-01 00:18:00',
               '2022-01-01 00:34:00', '2022-01-01 00:35:00',
               ...
               '2022-01-01 16:15:00', '2022-01-01 16:16:00',
               '2022-01-01 16:20:00', '2022-01-01 16:21:00',
               '2022-01-01 16:25:00', '2022-01-01 16:34:00',
               '2022-01-01 16:36:00', '2022-01-01 16:37:00',
               '2022-01-01 16:39:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=387, freq=None)
| dt                  |   open |   high |    low |   close |   volume |
|:--------------------|-------:|-------:|-------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 100.25 |  99.75 |  100    |        4 |
| 2022-01-01 00:15:00 |  99.75 |  99.75 |  99.25 |   99.25 |        1 |
| 2022-01-

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:05:00', '2022-01-01 00:08:00',
               '2022-01-01 00:12:00', '2022-01-01 00:15:00',
               '2022-01-01 00:17:00', '2022-01-01 00:18:00',
               '2022-01-01 00:34:00', '2022-01-01 00:35:00',
               ...
               '2022-01-01 16:15:00', '2022-01-01 16:16:00',
               '2022-01-01 16:20:00', '2022-01-01 16:21:00',
               '2022-01-01 16:25:00', '2022-01-01 16:34:00',
               '2022-01-01 16:36:00', '2022-01-01 16:37:00',
               '2022-01-01 16:39:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=387, freq=None)
DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:05:00', '2022-01-01 00:08:00',
               '2022-01-01 00:12:00', '2022-01-01 00:15:00',
               '2022-01-01 00:17:00', '2022-01-01 00:18:00',
               '2022-01-01 00:34:00', '2022-01-01 00:3

### Plot the results

In [9]:
from source.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:02:00',
               '2022-01-01 00:05:00', '2022-01-01 00:08:00',
               '2022-01-01 00:12:00', '2022-01-01 00:15:00',
               '2022-01-01 00:17:00', '2022-01-01 00:18:00',
               '2022-01-01 00:34:00', '2022-01-01 00:35:00',
               ...
               '2022-01-01 16:15:00', '2022-01-01 16:16:00',
               '2022-01-01 16:20:00', '2022-01-01 16:21:00',
               '2022-01-01 16:25:00', '2022-01-01 16:34:00',
               '2022-01-01 16:36:00', '2022-01-01 16:37:00',
               '2022-01-01 16:39:00', '2022-01-01 16:40:00'],
              dtype='datetime64[ns]', name='dt', length=387, freq=None)


In [10]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()